### Notebook for Image Preprocessing
```
Used for comparing, and visualizing various preprocessing steps
```

In [ ]:
import glob
import os
import sys

import cv2
import easyocr
from matplotlib import pyplot as plt

from licenseplates.model import lpmodel

EXAMPLE_IMAGE_PATH = "License-Plates-5/test/images"
RESULT_IMAGE_PATH = "License-Plates-5/output"

In [ ]:
def save_images(curr_img_index, images, titles):
    full_path = os.path.join(RESULT_IMAGE_PATH, f"img-{curr_img_index}")
    if not os.path.exists(full_path):
        os.makedirs(full_path)

    for image, title in zip(images, titles): 
        updated_title = title.replace(" ", "_").lower() + ".jpg"
        img_path = os.path.join(full_path, updated_title)
        print(img_path)
        cv2.imwrite(img_path, image)

In [ ]:
STORE_IMAGES = True

In [ ]:
test_img_paths = glob.glob(os.path.join(EXAMPLE_IMAGE_PATH, "*.jpg"))

img_select_idx = 22
img_path = test_img_paths[img_select_idx]

print("Index of current image: ", img_select_idx)
test_image = cv2.imread(img_path)

bounding_box_data = lpmodel.predict_bounding_box(image=test_image, model=lpmodel.load_model())
x1, y1, x2, y2 = bounding_box_data["bb"]
bb_image = test_image.copy()
cv2.rectangle(bb_image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 1)
plt.imshow(bb_image)
plt.show()

reader = easyocr.Reader(['en'])
result = reader.readtext(bb_image)
print("Original Image LP: ", result)

border_edges, border_rect = lpmodel.detect_borders(bounding_box_data)
border_image = bounding_box_data["cropped_img"].copy()
cv2.drawContours(border_image, [border_edges], 0, (0, 255, 0), 2)
plt.imshow(border_image)
plt.show()

transformed_image = lpmodel.transform_license_plate(image=bounding_box_data["cropped_img"], rectangle=border_rect, corners=border_edges)
plt.imshow(transformed_image)
plt.show()

result = reader.readtext(transformed_image)
print("Transformed Image LP: ", result)

lp_text, images, titles = lpmodel.read_license_plate(image=transformed_image)
print("Preprocessed Image LP: ", lp_text)

if STORE_IMAGES is True:
    save_images(img_select_idx, images, titles)
    STORE_IMAGES = False

```
Plot preprocessing steps
```

In [ ]:
def plot_images(images, titles):
    num_images = len(images)

    # Create a subplot grid based on the number of images
    rows = 2  # You can adjust the number of rows and columns based on your preference
    cols = (num_images + 1) // rows

    fig, axes = plt.subplots(rows, cols, figsize=(12, 8))

    # Flatten the axes if there is only one row
    if rows == 1:
        axes = axes.reshape(1, -1)

    for i in range(num_images):
        axes[i // cols, i % cols].imshow(images[i], cmap='gray')
        axes[i // cols, i % cols].set_title(titles[i])
        axes[i // cols, i % cols].axis('off')

    plt.tight_layout()
    plt.show()

plot_images(images=images, titles=titles)